In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_id = "gmguarino/camembertav2-base-climateguard"

model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained("almanach/camembertav2-base")
model.eval()

In [ ]:
from datasets import load_dataset

dataset_id = "DataForGood/climateguard"
# Load raw dataset
dataset = load_dataset(dataset_id, split="test")

# Tokenize helper function
def tokenize(batch):
    return tokenizer(
        batch["plaintext"], padding=True, truncation=True, return_tensors="pt"
    )


def format_labels(example):
    return {"label": int(example["misinformation"])}


# Tokenize dataset
# if "misinformation" in split_dataset["train"].features.keys():
#     split_dataset =  split_dataset.rename_column("misinformation", "labels") # to match Trainer
tokenized_dataset = dataset.map(
    tokenize, batched=True
)
tokenized_dataset = tokenized_dataset.map(format_labels, batched=False)



In [ ]:
import torch
from tqdm.notebook import tqdm

predictions = []
labels = []

with torch.no_grad():
    # for record in tokenized_dataset:
    inputs = tokenizer(dataset["plaintext"], return_tensors="pt", padding="longest", truncation=True)
    outputs = model(inputs["input_ids"], inputs["attention_mask"])
    print(outputs.logits.numpy().argmax())
    # predictions.append(outputs.logits.numpy().argmax())
    # labels.append(int(dataset["misinformation"]))

In [ ]:
labels = outputs.logits.argmax(1)
predictions = list(map(int, dataset["misinformation"]))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

In [ ]:
              precision    recall  f1-score   support

           0       0.85      0.65      0.74        97
           1       0.24      0.50      0.33        22

    accuracy                           0.62       119
   macro avg       0.55      0.57      0.53       119
weighted avg       0.74      0.62      0.66       119
